In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from dateutil.parser import parse
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset
import matplotlib.pyplot as plt

In [2]:
###TODO: expand to full data, figure out how to visualize the difference in compounds or just train multiple neural networks on each compound, fix the size error idk why thats happening

In [3]:
filenames = ["LapWeather_Australian Grand Prix.csv","LapWeather_Austrian Grand Prix.csv","LapWeather_Azerbaijan Grand Prix.csv","LapWeather_Bahrain Grand Prix.csv","LapWeather_Belgian Grand Prix.csv","LapWeather_Brazilian Grand Prix.csv","LapWeather_British Grand Prix.csv","LapWeather_Canadian Grand Prix.csv", "LapWeather_Chinese Grand Prix.csv","LapWeather_French Grand Prix.csv","LapWeather_German Grand Prix.csv","LapWeather_Hungarian Grand Prix.csv","LapWeather_Italian Grand Prix.csv","LapWeather_Japanese Grand Prix.csv","LapWeather_Mexican Grand Prix.csv","LapWeather_Monaco Grand Prix.csv","LapWeather_Russian Grand Prix.csv","LapWeather_Singapore Grand Prix.csv","LapWeather_Spanish Grand Prix.csv","LapWeather_United States Grand Prix.csv"]
combined_csv = pd.concat([pd.read_csv(f) for f in filenames ])
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [15]:
df = pd.read_csv('combined_csv.csv')
df = df[df["IsAccurate"]==True]
print(len(df))
df = df[df["TrackStatus"]==1]
print(len(df))
df = df.drop(['Time','DriverNumber','LapStartDate', 'Unnamed: 0', 'PitOutTime', 'PitInTime', 'Sector1Time','Sector2Time','Sector3Time','Sector1SessionTime','Sector2SessionTime','Sector3SessionTime','SpeedI1','SpeedI2','SpeedST','IsPersonalBest','FreshTyre','SpeedFL','LapStartTime','Driver','TrackStatus','IsAccurate'], axis=1)
df = df.dropna()

dfOutput = df['LapTime']
train_targets = dfOutput.to_numpy()

for i in range(len(train_targets)):
    train_targets[i] = train_targets[i].replace('0 days ', '')
# print(train_targets)
actual_train_targets = []
for time in train_targets:
    td = parse(time) - parse('00:00:00')
    seconds = td.total_seconds()
    actual_train_targets.append(seconds)
# print(actual_train_targets)
dfLapTime = pd.DataFrame(actual_train_targets)

df['LapTime'] = dfLapTime
df = df.dropna()
cols = ['LapTime']
Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3-Q1
df = df[(~(df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
dfInput = df.drop(['LapTime'], axis=1)
dfInput = dfInput.replace({'SUPERHARD':1, 'HARD':2, 'MEDIUM':3, 'SOFT':4,'SUPERSOFT': 5, 'ULTRASOFT': 6,'HYPERSOFT':7,"INTERMEDIATE":0,"WET":-1})
dfInput = dfInput.replace({'Ferrari':1,'Mercedes':2,'Red Bull Racing':3, 'McLaren':4, 'Renault':5, 'Force India':6, 'Sauber':7, 'Williams':8, 'Toro Rosso':9, 'Haas F1 Team':10, 'Racing Point':6})
train_inputs = dfInput.to_numpy()
inputs_array = train_inputs.astype('float64')
dfOutput = df['LapTime']

targets_array = dfOutput.to_numpy()
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

new_shape = (len(targets_array), 1)
targets = targets.view(new_shape)
print(targets.shape)

print(inputs_array)
print(targets_array)


18288
17264
torch.Size([13743, 1])
[[  2.    6.    4.  ...  38.2 296.    3.8]
 [  3.    6.    5.  ...  36.7 289.    4.3]
 [  4.    6.    6.  ...  36.8 255.    2.9]
 ...
 [ 65.    6.   52.  ...  34.3 212.    1.4]
 [ 66.    6.   53.  ...  34.1 213.    1.5]
 [ 67.    6.   54.  ...  34.3 159.    1.7]]
[ 90.177  89.61   89.54  ... 100.552 100.83  103.413]


In [16]:
dataset = TensorDataset(inputs, targets)

In [17]:
val_percent = 0.2
num_rows = len(dfInput.index)
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [18]:
batch_size = 10
train_loader = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_ds, batch_size, num_workers = 0)

In [19]:
input_size = len(inputs[0])
output_size = 1

In [109]:
class MultiLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(12, 36),
            nn.ReLU(),
            nn.Linear(36,18),
            nn.ReLU(),
            nn.Linear(18,1)
        )
        
    def forward(self, xb):
        out = self.layers(xb)                       
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        mse_loss = nn.MSELoss()
        # cross_entropy_loss = nn.CrossEntropyLoss()
        # loss = cross_entropy_loss(out, targets)
        loss = mse_loss(out, targets)  
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [110]:
model = MultiLinearRegressionModel()

In [ ]:
# list(model.parameters())

In [111]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [112]:
result = evaluate(model, val_loader)
print(result)

{'val_loss': 84.27987670898438}


In [113]:
epochs = 200
lr = 1e-7
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 84.2799
Epoch [40], val_loss: 84.2799


KeyboardInterrupt: 

In [83]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    return target, prediction

In [84]:
input, target = val_ds[12]
predict_single(input, target, model)

(tensor([95.4360]), tensor([91.5031]))

In [85]:
input, target = val_ds[50]
predict_single(input, target, model)

(tensor([78.8990]), tensor([88.5105]))

In [86]:
def get_accuracy(predict,actual):
        predict = predict.detach().numpy()
        actual = actual.numpy()

        count = 0

        for i in range(len(actual)):
            if predict[i] >= actual[i]*.9 and predict[i] <= actual[i]*1.1:
                count+=1
        return count/len(predict)*100


In [87]:
targets = []
predictions = []
for i in range(len(val_ds)):
    input, target = val_ds[i]
    target, prediction = predict_single(input, target, model)
    # np.append(targets,target)
    targets.append(target)
    predictions.append(prediction)
    # np.append(predictions,prediction)

# print(len(predictions))
# print(targets[0])
accuracy = []
for j in range(len(predictions)-1):
    accuracy.append(get_accuracy(predictions[j],targets[j]))

In [88]:
print(np.sum(accuracy)/len(accuracy))

54.05897342555515


In [ ]:
# fig, ax = plt.subplots(figsize=(20, 10))

# ax.scatter(range(len(predictions)),targets)
# ax.scatter(range(len(predictions)), predictions)
